In [ ]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings(action='ignore')

np.random.seed(100)



def load_data():
    
    iris = load_iris()
    X = iris.data[:, [2, 3]]
    y = iris.target



    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=0)
    
    return train_X, test_X, train_y, test_y
    
"""


1. 분류를 위한 의사결정 나무 모델을 정의하고

   모델 내부의 파라미터들을 정의합니다.
   
   Step01. 분류를 위한 의사결정 나무 모델을 정의합니다.
           random_state는 0으로 설정합니다.
   
   Step02. param_grid의 criterion, max_depth, 
           min_samples_split에 원하는 다양한 값을 딕셔너리 형태로 
           넣어줍니다. 각 인자의 값은 1차원 리스트여야 합니다.
           
   Step03. grid_dtree 변수를 GridSearchCV로 정의하고,
           GridSearchCV 안에서 refit과 return_train_score은
           각각 True로 설정합니다.
           
   Step04. grid_dtree를 학습시킵니다.
   
"""



def DT_clf(train_X, train_y):  
    
    iris_tree = DecisionTreeClassifier(random_state=0)
    
    param_grid = {'criterion' : ['entropy', 'gini'], 
                  'max_depth' : [1,2,3,4,5], 
                  'min_samples_split' : [2,3]}
    
    grid_dtree = GridSearchCV(iris_tree, 
                              param_grid = param_grid,
                              refit= True, 
                              return_train_score = True)
                            
    grid_dtree.fit(train_X, train_y)
                                  
    return grid_dtree


    
"""

2. 다양한 인자를 설정한 후 학습시킨 분류 의사결정 
   나무 모델들 중, 최고 성능을 가진 모델의

   파라미터 목록과 정확도를 살펴봅시다. 
      
   Step01. 이미 구현된 함수를 통해 데이터를 불러옵니다.
   
   Step02. 구현된 DT_clf 함수의 속성을 사용하여
           최적의 파라미터와 최고 정확도를
           얻고 출력해봅니다.
           
   Step03. 구현된 DT_clf 함수의 속성을 사용하여
           최적의 파라미터로 학습된 estimator를
           얻어봅니다.
           
   Step04. 최적의 파라미터로 학습된 estimator를
           사용하여 테스트 데이터에 대한 분류 결과를
           예측합니다.
           
   Step05. 테스트 데이터에 대한 정확도를 확인해봅시다.
           우리의 목표는 적절한 파라미터 조정을 통해 테스트
           데이터에 대한 정확도가 0.97이상이 나오도록 하는 것입니다.


"""


def main():
    
    train_X, test_X, train_y, test_y = load_data()
    
    model = DT_clf(train_X, train_y)
    
    best_p = model.best_params_
    best_score = model.best_score_
    
    print('GridSearchCV 최적 파라미터 : ', best_p)
    print('GridSearchCV 최고 정확도 : {0:.4f}'.format(best_score))


    estimator = model.best_estimator_


    pred_y = estimator.predict(test_X)
    
    accuracy = accuracy_score(test_y, pred_y)
    
    print('Accuracy: %.5f' % accuracy)
    
    return accuracy
    
if __name__ == "__main__":
    main()

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

def load_data():
    # iris 데이터셋을 불러옵니다.

    dataset = load_iris()
    
    # <ToDo>: feature 데이터를 불러옵니다.
    x = dataset.data
    # <ToDo>: label 데이터를 불러옵니다.
    y = dataset.target
    
    # <ToDo>: feature 데이터의 변수 이름을 불러옵니다. 
    feature_name = dataset.feature_names
    # <ToDo>: label 데이터의 변수 이름을 불러옵니다.
    label_name = dataset.target_names
    
    # <ToDo>: 불러온 데이터 중 70%는 학습용, 30%는 테스트용으로 활용합니다. 
    # 이때 random_state는 81로 설정합니다.
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=81)
    
    return x_train, y_train, x_val, y_val, feature_name, label_name

In [ ]:
# main.py
from elice_utils import EliceUtils
elice_utils = EliceUtils()

import preprocess
import DT
import plot_graph

def main():
    # 데이터를 불러옵니다.

    x_train, y_train, x_val, y_val, feature_name, label_name = preprocess.load_data()
    
    # <ToDo>: DT.py 안의 함수를 사용해, 최고의 성능을 보이는
    #         모델을 선정하여 훈련용 데이터로 학습시킵니다.
    
    clf = DT.select_model(x_train, y_train)
    trained_clf = DT.train_model(clf, x_train, y_train)
    
    # confusion matrix를 출력합니다.
    plot_graph.plot_confusion_matrix(trained_clf, x_val, y_val)
    
    # <ToDo>: DT.py 안의 함수를 사용해, 훈련된 모델의 예측 성능
    #         (F1-Score)을 검증용 데이터로 계산하고 출력합니다.
    
    F1_Score = DT.evaluate_model(trained_clf, x_val, y_val)
    print("F1 Score: %0.2f" % (F1_Score))
    
    # <ToDo>: DT.py 안의 함수를 사용해, 모델 학습 중 독립 변수의
    #         중요도를 정의하고, 그래프를 통해 확인합니다.
    
    importance = DT.var_importance(trained_clf)
    plot_graph.importance_barplot(trained_clf, importance)
    
    # 학습된 모델로부터 의사결정 나무의 분류 경계면과 모델의 구조를 출력합니다.
    plot_graph.model_plot(trained_clf, x_train, y_train, feature_name, label_name)
    
    return F1_Score


if __name__ == "__main__":
    main()

In [ ]:
# DT.py

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.model_selection import GridSearchCV


def select_model(feature, label):
    
    # <ToDo>: 사용할 모델을 지시사항에 따라 지정합니다.
    estimator = DecisionTreeClassifier(random_state=81)
    
    # <ToDo>: estimator의 튜닝을 위한 파라미터를 지시사항에 따라 지정합니다.
    param_grid = {'max_depth': [2,3,4,5]}
    
    # <ToDo>: 최적 파라미터로 다시 estimator를 학습시킬지 말지를 지시사항에 따라 지정합니다.
    refit = True
    
    # <ToDo>: k-fold 교차 검증의 k를 지시사항에 따라 지정합니다. 
    k = 5
    
    # <ToDo>: 성능 평가 방법을 지시사항에 따라 지정합니다.
    scoring = 'f1_micro'
    
    # GridSearchCV의 인자들을 채워넣습니다.
    selected_model = GridSearchCV(estimator = estimator, 
                                  param_grid = param_grid,
                                  refit = refit, 
                                  cv = k,
                                  scoring = scoring)
    
    return selected_model




def train_model(model, feature, label):
    
    # <ToDo>: 받아온 모델을 feature, label로 학습시킵니다.
    trained_model = model.fit(feature, label)
    
    # <ToDo>: 학습된 모델들 중 최적의 모델을 찾아냅니다.
    best_model = trained_model.best_estimator_


    return best_model



def var_importance(best_model):
    
    # <ToDo>: 학습된 모델들 중 최적의 모델에 사용된 독립변수의 중요도를 리턴합니다.
    return best_model.feature_importances_




def evaluate_model(model, feature, label):
    
    # <ToDo>: 검증용으로 주어진 데이터를 이용해서 모델의 성능을 평가합니다.
    f1 = model.score(feature, label)
    
    return f1


In [ ]:
# preprocess.py

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

def load_data():
    # diabetes 데이터셋을 불러옵니다.

    dataset = load_diabetes()
    
    # <ToDo>: feature 데이터를 불러옵니다.
    x = dataset.data
    # <ToDo>: label 데이터를 불러옵니다.
    y = dataset.target
    
    # <ToDo>: feature 데이터의 변수 이름을 불러옵니다. 
    feature_name = dataset.feature_names
    
    # <ToDo>: 불러온 데이터 중 75%는 학습용, 25%는 테스트용으로 활용합니다. 
    # 이때 random_state는 81로 설정합니다.
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.25, random_state=81)
    
    return x_train, y_train, x_val, y_val, feature_name


In [ ]:
#  main.py

from elice_utils import EliceUtils
elice_utils = EliceUtils()

import preprocess
import DT
import plot_graph

def main():
    # 데이터를 불러옵니다.
    x_train, y_train, x_val, y_val, feature_name = preprocess.load_data()

    # <ToDo>: DT.py 안의 함수를 사용해, 최고의 성능을 보이는

    #         모델을 선정하여 훈련용 데이터로 학습시킵니다.
    
    clf = DT.select_model(x_train, y_train)
    trained_clf = DT.train_model(clf, x_train, y_train)


    # <ToDo>: DT.py 안의 함수를 사용해, 훈련된 모델의 예측 성능

    #         (MSE)을 검증용 데이터로 계산하고 출력합니다.
    
    MSE = DT.evaluate_model(trained_clf, x_val, y_val)
    print("MSE: %0.2f" % (MSE))


    # <ToDo>: DT.py 안의 함수를 사용해, 모델 학습 중 독립 변수의

    #         중요도를 정의하고, 그래프를 통해 확인합니다.
    
    importance = DT.var_importance(trained_clf)
    plot_graph.importance_barplot(trained_clf, importance)


    # 학습된 모델로부터 의사결정 나무의 분류 경계면과 모델의 구조를 출력합니다.

    plot_graph.model_plot(trained_clf, x_train[:,4:], y_train , feature_name[4:])
    
    return MSE


if __name__ == "__main__":
    main()

In [ ]:
# DT.py

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV


def select_model(feature, label):
    
    # <ToDo>: 사용할 모델을 지시사항에 따라 지정합니다.
    estimator = DecisionTreeRegressor(random_state=81)
    
    # <ToDo>: estimator의 튜닝을 위한 파라미터를 지시사항에 따라 지정합니다.
    param_grid = {'max_depth': [2,3,4,5,6,7,8,9,10,11]}
    
    # <ToDo>: 최적 파라미터로 다시 estimator를 학습시킬지 말지를 지시사항에 따라 지정합니다.
    refit = True
    
    # <ToDo>: k-fold 교차 검증의 k를 지시사항에 따라 지정합니다. 
    k = 5
    
    # <ToDo>: 성능 평가 방법을 지시사항에 따라 지정합니다.
    scoring = 'neg_mean_squared_error'
    
    # GridSearchCV의 인자들을 채워넣습니다.
    selected_model = GridSearchCV(estimator = estimator, 
                                  param_grid = param_grid,
                                  refit = refit, 
                                  cv = k,
                                  scoring = scoring)
    
    return selected_model




def train_model(model, feature, label):
    
    # <ToDo>: 받아온 모델을 feature, label로 학습시킵니다.
    trained_model = model.fit(feature, label)
    
    # <ToDo>: 학습된 모델들 중 최적의 모델을 찾아냅니다.
    best_model = trained_model.best_estimator_


    return best_model



def var_importance(best_model):
    
    # <ToDo>: 학습된 모델들 중 최적의 모델에 사용된 독립변수의 중요도를 리턴합니다.
    return best_model.feature_importances_




def evaluate_model(model, feature, label):
    
    # <ToDo>: 검증용으로 주어진 데이터를 이용해서 모델의 성능을 평가합니다.
    mse = model.score(feature, label)
    
    return mse